In [18]:
# import shutil
# shutil.copytree(r'/kaggle/input/i-am-a-painter-github', r'/kaggle/working')

import os
os.listdir('I-am-a-painter')

['test.py',
 '.git',
 'README.md',
 'requirements.txt',
 'main.ipynb',
 '.gitignore',
 'model',
 'dataLoader.py']